<a href="https://colab.research.google.com/github/fedefliguer/trading/blob/master/v1_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalación librerías

In [1]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 6.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=5bab45693c8c55e7ca1ba7d1804e31a4b5c222e289883302f2d685fb58593a59
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


## Detalle de funciones

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
pd.options.mode.chained_assignment = None
from datetime import date
from datetime import timedelta 
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import fbeta_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 1000)

In [3]:
def descarga(ticker, fc_empieza, fc_termina):
  base = yf.download(ticker, start=fc_empieza, end=fc_termina)
  base = base[['Close', 'Volume', 'High', 'Low']]
  base.insert(loc=0, column='Ticker', value=ticker)
  base.reset_index(level=0, inplace=True)
  base.columns=['fc', 'ticker', 'y', 'vl', 'high', 'low']
  return base

In [4]:
def calcula_pc_merval(dataset):
  dataset = pd.merge(dataset,mvl,on='fc',how='left')
  dataset['pc_merval'] = dataset.y/dataset.mvl
  dataset = dataset.drop(['mvl'], axis=1)
  return dataset

In [5]:
def calcula_amplitud(dataset):
  dataset['amplitud'] = (dataset.high - dataset.low)/dataset.y
  return dataset

In [6]:
def estandariza_volumen(dataset):
  mean_vl = dataset['vl'].mean()
  std_vl = dataset['vl'].std()
  dataset['vl'] = (dataset.vl - mean_vl)/std_vl
  return dataset

In [7]:
def calcula_medias(dataset):
  period = 12
  sma = dataset['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = dataset['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  dataset['exp1'] = ema
  period = 26
  sma = dataset['y'].rolling(period, min_periods=period).mean()
  idx_start = sma.isna().sum() + 1 - period
  idx_end = idx_start + period
  sma = sma[idx_start: idx_end]
  rest = dataset['y'][idx_end:]
  ema = pd.concat([sma, rest]).ewm(span=period, adjust=False).mean()
  dataset['exp2'] = ema
  macd = dataset['exp1']-dataset['exp2']
  dataset['macd'] = macd
  dataset['exp3'] = macd.ewm(span=9, adjust=False).mean()
  dataset['histog'] = dataset['macd'] - dataset['exp3'] 
  return dataset

In [8]:
def calcula_historia(dataset, lags):
  for (columnName, columnData) in dataset.iloc[:,6:].iteritems():
    i = 1
    while i < lags:
      colname = "var_%s_%s" % (columnName, i)
      dataset[colname] = columnData/columnData.shift(i)-1
      i = i + 1
  return dataset

In [9]:
def calcula_canalidad_y(dataset):
  i = 1
  dataset['lag_y_1'] = dataset.y.shift(1)
  dataset['nu_dias_y_entre_max_min_30'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_30'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset['nu_dias_y_entre_max_min_90'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_90'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset['nu_dias_y_entre_max_min_180'] = np.where((dataset['lag_y_1'] < dataset['high']) & (dataset['lag_y_1'] > dataset['low']), 1, 0)
  dataset['nu_dias_y_entre_5pc_180'] = np.where((dataset['lag_y_1'] < (dataset.y * 1.05)) & (dataset['lag_y_1'] > (dataset.y * 0.95)), 1, 0)

  dataset = dataset.drop(['lag_y_1'], axis=1)
  i = 2
  while i < 30:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_30'] = dataset['nu_dias_y_entre_max_min_30'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_30'] = dataset['nu_dias_y_entre_5pc_30'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  i = 2
  while i < 90:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_90'] = dataset['nu_dias_y_entre_max_min_90'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_90'] = dataset['nu_dias_y_entre_5pc_90'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  i = 2
  while i < 180:
    colname = "lag_y_%s" % (i)
    dataset[colname] = dataset.y.shift(i)
    dataset['nu_dias_y_entre_max_min_180'] = dataset['nu_dias_y_entre_max_min_180'] + np.where((dataset[colname] < dataset['high']) & (dataset[colname] > dataset['low']), 1, 0)
    dataset['nu_dias_y_entre_5pc_180'] = dataset['nu_dias_y_entre_5pc_180'] + np.where((dataset[colname] < (dataset.y * 1.05)) & (dataset[colname] > (dataset.y * 0.95)), 1, 0)
    i = i + 1
    dataset = dataset.drop([colname], axis=1)

  return dataset

In [10]:
def calcula_canalidad_histog_macd(dataset):
  list = [5, 30, 90, 180]
  for ventana in list:
    i = 1
    dataset['lag_histog_1'] = dataset.histog.shift(1)
    colname_nu_1 = "nu_dias_histog_entre_5pc_%s" % (ventana)
    dataset[colname_nu_1] = np.where((dataset['lag_histog_1'] < (dataset.histog * 1.05)) & (dataset['lag_histog_1'] > (dataset.histog * 0.95)), 1, 0)

    colname_nu_2 = "nu_dias_histog_positivo_%s" % (ventana)
    dataset[colname_nu_2] = np.where((dataset['lag_histog_1']>0), 1, 0)

    colname_nu_3 = "nu_dias_histog_negativo_%s" % (ventana)
    dataset[colname_nu_3] = np.where((dataset['lag_histog_1']<0), 1, 0)

    colname_nu_4 = "nu_dias_histog_mismo_signo_%s" % (ventana)
    dataset[colname_nu_4] = np.where(((dataset['lag_histog_1']>0) & (dataset['histog']>0))|((dataset['lag_histog_1']<0) & (dataset['histog']<0)), 1, 0)

    dataset = dataset.drop(['lag_histog_1'], axis=1)
    i = 2
    while i < (ventana+1):
      colname = "lag_histog_%s" % (i)
      dataset[colname] = dataset.histog.shift(i)
      dataset[colname_nu_1] = dataset[colname_nu_1] + np.where((dataset[colname] < (dataset.histog * 1.50)) & (dataset[colname] > (dataset.histog * 0.50)), 1, 0)
      dataset[colname_nu_2] = dataset[colname_nu_2] + np.where((dataset[colname]>0), 1, 0)
      dataset[colname_nu_3] = dataset[colname_nu_3] + np.where((dataset[colname]<0), 1, 0)
      dataset[colname_nu_4] = dataset[colname_nu_4] + np.where(((dataset[colname]>0) & (dataset['histog']>0))|((dataset[colname]<0) & (dataset['histog']<0)), 1, 0)
      i = i + 1
      dataset = dataset.drop([colname], axis=1)
  return dataset

In [11]:
def calcula_AT_tendencias(dataset, lags):
  
  # Construye las columnas para determinar si es un pico
  i = 1
  while i < (lags+1):
      colname = 'p%sb' % (i)                                                  
      dataset[colname] = round(dataset.y.shift(i),2)
      j = i * -1
      colname = 'p%sf' % (-j)                                                  
      dataset[colname] = round(dataset.y.shift(j),2)
      i = i + 1

  # Determina si es un pico  
  dataset['maxb'] = round(dataset.filter(regex=(".*b")).max(axis=1),2)
  dataset['maxf']= round(dataset.filter(regex=(".*f")).max(axis=1),2)
  dataset['minb'] = round(dataset.filter(regex=(".*b")).min(axis=1),2)
  dataset['minf'] = round(dataset.filter(regex=(".*f")).min(axis=1),2)
  dataset['T'] = np.where((dataset['y']>dataset['maxb']) & (dataset['y']>dataset['maxf']), 1, 0)
  dataset['P'] = np.where((dataset['y']<dataset['minb']) & (dataset['y']<dataset['minf']), 1, 0)

  techos = dataset[(dataset['T']==1)]
  techos['m'] = (techos.y.shift(1) - techos.y)/(techos.fc.shift(1) - techos.fc).dt.days
  techos.name = 'techos'
  pisos = dataset[(dataset['P']==1)]
  pisos['m'] = (pisos.y.shift(1) - pisos.y)/(pisos.fc.shift(1) - pisos.fc).dt.days
  pisos.name = 'pisos'
  dataset_list = [techos, pisos]

  for dataset_picos in dataset_list:  # En cada dataset (techos y pisos)
    name = dataset_picos.name
    dias = len(dataset)
    for index, row in dataset_picos.iloc[1:].iterrows(): # Para cada pico detectado (fila del dataset) a partir del segundo (porque el primero no tiene anterior, no tiene tendencia)
      y_start = row['y']
      pendiente = row['m']
      if (dias < np.where(dataset.fc==row['fc'])[0] + lags):
        continue    
      serie = [] # Crea la serie que va a contener el precio proyectado
      serie = np.append(serie, np.repeat(np.nan, (np.where(dataset.fc==row['fc'])[0] + lags))) # Appendea nulos hasta el día en el que confirmamos que nació una tendencia
      i = np.where(dataset.fc==row['fc'])[0] + lags
      while (i < dias):
        dia = i - (np.where(dataset.fc==row['fc'])[0] + lags)
        serie = np.append(serie, (y_start + pendiente*lags) + pendiente*dia)
        i = i + 1 # Appendea el precio proyectado hasta el final

      colname = '%s_%s_proy' % (name, index)  # Precio proyectado
      dataset[colname] = serie # Construye la columna de toda la serie

      # Construyo columna con veces en la que el pico fue superado
      colname_pass = '%s_%s_pass' % (name, index) # Pico pasado
      if name == 'techos':
        dataset[colname_pass] = np.where(dataset['y']>(dataset[colname])*1.005, 1, 0)
      elif name == 'pisos':
        dataset[colname_pass] = np.where(dataset['y']<(dataset[colname])*0.995, 1, 0)
      dataset[colname_pass] = dataset[colname_pass].cumsum()

      # Construyo columna con veces en la que el pico fue probado
      colname_prueba = '%s_%s_prueba' % (name, index)  
      dataset[colname_prueba] = np.where((dataset['y']>dataset[colname]*0.995)&(dataset['y']<dataset[colname]*1.005), 1, 0)
      dataset[colname_prueba] = dataset[colname_prueba].cumsum()

      # Construyo columna con pendiente del pico
      colname_pendiente = '%s_%s_pendiente' % (name, index)  
      dataset[colname_pendiente] = row['m']

      # Creo la combinacion y elimino cada uno
      colname_comb = '%s_%s' % (name, index)
      dataset[colname_comb] = dataset[[colname, colname_pass, colname_prueba, colname_pendiente]].values.tolist()
      del dataset[colname]
      del dataset[colname_pass]
      del dataset[colname_prueba]
      del dataset[colname_pendiente]

  # Creo el objeto por cada techo o piso individual
  names_techos = dataset.filter(regex=("(techos)(.*)")).columns
  names_pisos = dataset.filter(regex=("(pisos)(.*)")).columns

  for index, row in dataset.iterrows():  # Por cada fila del dataset original (por cada precio)

    # Genero las rows vacías con las variables agregadas
    nu_pruebas_techo_vivo_mas_probado = np.nan    
    precio_proyectado_techo_vivo_mas_probado = np.nan
    precio_proyectado_techo_vivo_mas_cercano = np.nan
    precio_proyectado_techo_muerto_mas_cercano = np.nan
    tendencia_techo_vivo_mas_probado = np.nan

    nu_pruebas_piso_vivo_mas_probado = np.nan
    precio_proyectado_piso_vivo_mas_probado = np.nan
    precio_proyectado_piso_vivo_mas_cercano = np.nan
    precio_proyectado_piso_muerto_mas_cercano = np.nan
    tendencia_piso_vivo_mas_probado = np.nan

    # Voy a recorrer cada tendencia proyectada para definir cuáles van, en caso de que corresponda lo asigno a estas variables agregadas

    i = 0
    while i < len(row.index): # Por cada uno de los picos de los que se puede armar tendencia
      if (row.index[i] in names_techos):  # Si es un techo
        if row[i][1]>5: # Si está muerto
          if abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_techo_muerto_mas_cercano) or np.isnan(precio_proyectado_techo_muerto_mas_cercano): # Si está muerto y proyecta precio más cercano que el actual
            precio_proyectado_techo_muerto_mas_cercano = row[i][0]
            
        else: # Si está vivo
          if row[i][2] > nu_pruebas_techo_vivo_mas_probado or (np.isnan(nu_pruebas_techo_vivo_mas_probado) and row[i][2]>0): # Si fue más probado que el actual
            nu_pruebas_techo_vivo_mas_probado = row[i][2]
            precio_proyectado_techo_vivo_mas_probado = row[i][0]
            tendencia_techo_vivo_mas_probado = row[i][3]

          if (np.isnan(precio_proyectado_techo_vivo_mas_cercano)) or (abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_techo_vivo_mas_cercano)): # Si, sin haber muerto, proyecta un techo más alto que el actual
            precio_proyectado_techo_vivo_mas_cercano = row[i][0]

      elif (row.index[i] in names_pisos):
        if row[i][1]>5: # Si está muerto
          if abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_piso_muerto_mas_cercano) or np.isnan(precio_proyectado_piso_muerto_mas_cercano): # Si proyecta precio más cercano que el actual
            precio_proyectado_piso_muerto_mas_cercano = row[i][0]
            
        else: # Si está vivo
          if row[i][2] > nu_pruebas_piso_vivo_mas_probado or (np.isnan(nu_pruebas_piso_vivo_mas_probado) and row[i][2]>0): # Si fue más probado que el actual
            nu_pruebas_piso_vivo_mas_probado = row[i][2]
            precio_proyectado_piso_vivo_mas_probado = row[i][0]
            tendencia_piso_vivo_mas_probado = row[i][3]

          if (np.isnan(precio_proyectado_piso_vivo_mas_cercano)) or (abs(row['y']-row[i][0]) < abs(row['y']-precio_proyectado_piso_vivo_mas_cercano)): # Si, sin haber muerto, proyecta un techo más alto que el actual
            precio_proyectado_piso_vivo_mas_cercano = row[i][0]
      i = i + 1
        
    dataset.loc[index,'nu_pruebas_techo_vivo_mas_probado_'f"{lags}"] = nu_pruebas_techo_vivo_mas_probado
    dataset.loc[index,'precio_proyectado_techo_vivo_mas_probado_'f"{lags}"] = (precio_proyectado_techo_vivo_mas_probado - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_techo_vivo_mas_cercano_'f"{lags}"] = (precio_proyectado_techo_vivo_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_techo_muerto_mas_cercano_'f"{lags}"] = (precio_proyectado_techo_muerto_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'tendencia_techo_vivo_mas_probado_'f"{lags}"] = tendencia_techo_vivo_mas_probado/row['y']

    dataset.loc[index,'nu_pruebas_piso_vivo_mas_probado_'f"{lags}"] = nu_pruebas_piso_vivo_mas_probado
    dataset.loc[index,'precio_proyectado_piso_vivo_mas_probado_'f"{lags}"] = (precio_proyectado_piso_vivo_mas_probado - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_piso_vivo_mas_cercano_'f"{lags}"] = (precio_proyectado_piso_vivo_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'precio_proyectado_piso_muerto_mas_cercano_'f"{lags}"] = (precio_proyectado_piso_muerto_mas_cercano - row['y'])/row['y']
    dataset.loc[index,'tendencia_piso_vivo_mas_probado_'f"{lags}"] = tendencia_piso_vivo_mas_probado/row['y']

  # Elimino todas las que construí excepto estas
  i = 1
  while i < (lags+1):
      colname = 'p%sb' % (i)                                                  
      dataset = dataset.drop(colname, axis=1)
      j = i * -1
      colname = 'p%sf' % (-j)                                                  
      dataset = dataset.drop(colname, axis=1)
      i = i + 1

  ultimas_drop = ['maxb', 'maxf', 'minb', 'minf', 'T', 'P']
  dataset = dataset.drop(ultimas_drop, axis=1)
  dataset = dataset.drop(names_techos, axis=1)
  dataset = dataset.drop(names_pisos, axis=1)
  return dataset

In [12]:
def calcula_target_class(dataset, SL, TG, dias_indeterminacion):
  dataset['target'] = 99
  i = 1
  while i <= dias_indeterminacion:
    var_y_low = dataset.low.shift(-i)/df.y-1 # Variación del mínimo de cada día contra el precio de compra
    var_y_high = dataset.high.shift(-i)/df.y-1 # Variación del máximo de cada día contra el precio de compra
    target = np.where(var_y_low < -SL, 0, 99)
    target = np.where(var_y_high > TG, 1, target)
    dataset['target'] = np.where(dataset['target'] == 99 , target , dataset['target'])
    i = i + 1
#  df = df.iloc[:-dias_indeterminacion]  # Elimino las últimas filas que no llegan a tener target
  return dataset

In [13]:
def divide_dev_test(dataset, start_train, start_test, end_test):
  global df, x_dev, y_dev, x_test, y_test, x_val, y_val, df_test, df_dev
  month = dataset['fc'].dt.strftime('%Y%m')
  month = pd.to_numeric(month)
  if 'month' not in dataset:
    dataset.insert (1, "month", month)
  dataset = dataset[(dataset.target) < 90] # Elimina indeterminados

  df_dev = dataset[(dataset.month >= start_train) & (dataset.month < start_test)]
  df_test = dataset[(dataset.month >= start_test) & (dataset.month <= end_test)]

## Consolidado

In [14]:
dias_empieza = 5000
dias_termina = 200
today = date.today()
fc_empieza = today + timedelta(days=(dias_empieza*-1))
fc_termina = today + timedelta(days=(dias_termina*-1))

mvl = yf.download('^MERV', start=fc_empieza, end=fc_termina)
print("Descargado Merval")
mvl = mvl[['Close']]
mvl.reset_index(level=0, inplace=True)
mvl.columns=['fc','mvl']
base = pd.DataFrame()

for ticker in (
    'GGAL.BA',
    'BMA.BA',
    'BYMA.BA',
    'CEPU.BA',
    'COME.BA',
    'CRES.BA',
    'CVH.BA',
    'EDN.BA',
    'MIRG.BA',
    'PAMP.BA',
    'SUPV.BA',
    'TECO2.BA',
    'TGNO4.BA',
    'TGSU2.BA',
    'TRAN.BA',
    'VALO.BA',
    'YPFD.BA'
):
  df = descarga(ticker, fc_empieza, fc_termina) # (Días empieza, días termina)
  print("Descargado ", ticker)
  df = calcula_pc_merval(df)
  df = calcula_amplitud(df)
  df = estandariza_volumen(df)
 
  df = calcula_medias(df)
  df = df.dropna()
  df = calcula_historia(df, 5) # (Lags)
  df = calcula_canalidad_y(df)
  df = calcula_canalidad_histog_macd(df)

  for per in (360, 120, 90, 60, 30, 15, 8, 4):
    df = calcula_AT_tendencias(df,per)
    print("Calculé AT para", ticker, "en lags de", per)
  
  df = calcula_target_class(df, 0.06, 0.14, 90) # (Stop loss, Take gain, Días para indeterminación)
  base = base.append(df)

[*********************100%***********************]  1 of 1 completed
Descargado Merval
[*********************100%***********************]  1 of 1 completed
Descargado  GGAL.BA
Calculé AT para GGAL.BA en lags de 360
Calculé AT para GGAL.BA en lags de 120
Calculé AT para GGAL.BA en lags de 90
Calculé AT para GGAL.BA en lags de 60
Calculé AT para GGAL.BA en lags de 30
Calculé AT para GGAL.BA en lags de 15
Calculé AT para GGAL.BA en lags de 8
Calculé AT para GGAL.BA en lags de 4
[*********************100%***********************]  1 of 1 completed
Descargado  BMA.BA
Calculé AT para BMA.BA en lags de 360
Calculé AT para BMA.BA en lags de 120
Calculé AT para BMA.BA en lags de 90
Calculé AT para BMA.BA en lags de 60
Calculé AT para BMA.BA en lags de 30
Calculé AT para BMA.BA en lags de 15
Calculé AT para BMA.BA en lags de 8
Calculé AT para BMA.BA en lags de 4
[*********************100%***********************]  1 of 1 completed
Descargado  BYMA.BA
Calculé AT para BYMA.BA en lags de 360
Calculé 

In [48]:
from google.colab import files
base.to_csv('v1.csv')
files.download('v1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>